# JR95-250129: qPCR Analysis Report - Full Details

## TODO ADD STYLING TO NOTEBOOK OR HTML FILE OUTPUT

## Summary

### Description

What the experiment is but more importantly emphasize the purpose.

Briefly describe the experimental design (SSCT).

### Results

What are the main results save details for later, overall good, overall bad.

### Conclusion

What the MINE conclusions at this point in the analysis.
What do I think my next steps are?
More importantly, what feedback am I looking for and seeking from the audience to 
pay particular attention to?

## Table of Content

## Code & Analysis

### Setup

In [53]:
import shutil
from pathlib import Path

import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange, Whisker, Legend, Range1d
from bokeh.palettes import Bright6, Bright7, Category10 
from bokeh.transform import factor_cmap, dodge
from bokeh.io import output_notebook
from bokeh.io import export_svgs, export_png

from dctap.libs.pandas import set_defaultoptions, display, displaydf_full
from dctap.qpcr.constants import QPCRPATHS
from dctap.qpcr.utils.core import *

In [3]:
# Set pandas settings
set_defaultoptions(pd, supresscopywarning=None)
output_notebook()

Loading BokehJS ...

### Reading in the data

#### Collecting and annotating the data

In [4]:
# Read and annotate data from different plates and combine them
experiment_id = "JR95-250129"
plate_ids = ["JR95-250129-plate1", "JR95-250129-plate2", "JR95-250129-plate3", "JR95-250129-plate4"]

dfs = []
for plate_id in plate_ids:
    dfs.append(get_plate_data(experiment_id, plate_id))

df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)

display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id
0,JR95-250129,A01,P6_D0-2DD_SD0.4_TR1_0hr,CER1,28.067467,JR95-250129-plate1
1,JR95-250129,A02,P6_D0-2DD_SD0.4_TR1_0hr,CER1,27.777831,JR95-250129-plate1
2,JR95-250129,A03,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.799317,JR95-250129-plate1
3,JR95-250129,A04,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.671907,JR95-250129-plate1
4,JR95-250129,A05,P6_D0-2DD_SD0.46_TR1_0hr,CER1,28.448518,JR95-250129-plate1
...,...,...,...,...,...,...
1015,JR95-250129,N12,P12_D2-6DD_SD0.4_TR2_48hr,GAL,22.041949,JR95-250129-plate4
1016,JR95-250129,N13,P12_D2-6DD_SD0.5_TR1_48hr,GAL,22.091856,JR95-250129-plate4
1017,JR95-250129,N14,P12_D2-6DD_SD0.5_TR1_48hr,GAL,22.323438,JR95-250129-plate4
1018,JR95-250129,N15,P12_D2-6DD_SD0.5_TR2_48hr,GAL,22.861958,JR95-250129-plate4


#### Demultiplex Sample labels and conditions

This will be helpful in dynamically assigning controls samples,
biological replicate samples, condition samples, and so on
for calculating foldchange based on the deltadeltacqMethod.

In [5]:
df_samples = get_sample_metadata(cast(pd.Series, df.Sample), sep="_")
with displaydf_full():
  display(df_samples)

,Sample,0,1,2,3,4
0,P6_D0-2DD_SD0.4_TR1_0hr,P6,D0-2DD,SD0.4,TR1,0hr
1,P6_D0-2DD_SD0.4_TR2_0hr,P6,D0-2DD,SD0.4,TR2,0hr
2,P6_D0-2DD_SD0.46_TR1_0hr,P6,D0-2DD,SD0.46,TR1,0hr
3,P6_D0-2DD_SD0.46_TR2_0hr,P6,D0-2DD,SD0.46,TR2,0hr
4,P12_D0-2DD_SD0.4_TR1_0hr,P12,D0-2DD,SD0.4,TR1,0hr
5,P12_D0-2DD_SD0.4_TR2_0hr,P12,D0-2DD,SD0.4,TR2,0hr
6,P12_D0-2DD_SD0.5_TR1_0hr,P12,D0-2DD,SD0.5,TR1,0hr
7,P12_D0-2DD_SD0.5_TR2_0hr,P12,D0-2DD,SD0.5,TR2,0hr
8,P6_D2-4DD_SD0.4_TR1_24hr,P6,D2-4DD,SD0.4,TR1,24hr
9,P6_D2-4DD_SD0.4_TR2_24hr,P6,D2-4DD,SD0.4,TR2,24hr


#### Adding helpful meta-data

In [6]:
conditions = ["bio_reps", "ctrl_calibrator", "cond_sd", "cond_chirtime"]
df = set_conditions(
    df,
    df_samples,
    conditions=conditions,
    merge_cols=["0124", "012", "02", "4"],
)

display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime
0,JR95-250129,A01,P6_D0-2DD_SD0.4_TR1_0hr,CER1,28.067467,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
1,JR95-250129,A02,P6_D0-2DD_SD0.4_TR1_0hr,CER1,27.777831,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
2,JR95-250129,A03,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.799317,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
3,JR95-250129,A04,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.671907,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
4,JR95-250129,A05,P6_D0-2DD_SD0.46_TR1_0hr,CER1,28.448518,JR95-250129-plate1,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr
...,...,...,...,...,...,...,...,...,...,...
1015,JR95-250129,N12,P12_D2-6DD_SD0.4_TR2_48hr,GAL,22.041949,JR95-250129-plate4,P12_D2-6DD_SD0.4_48hr,P12_D2-6DD_SD0.4,P12_SD0.4,48hr
1016,JR95-250129,N13,P12_D2-6DD_SD0.5_TR1_48hr,GAL,22.091856,JR95-250129-plate4,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr
1017,JR95-250129,N14,P12_D2-6DD_SD0.5_TR1_48hr,GAL,22.323438,JR95-250129-plate4,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr
1018,JR95-250129,N15,P12_D2-6DD_SD0.5_TR2_48hr,GAL,22.861958,JR95-250129-plate4,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr


#### Bulk Calculate deltaCq expression

In [7]:
df1 = get_deltaCq_expression_bulkdata(
    df,
    ref_primer="GAPDH",
    test_primers=get_primers(df),
    drop_customcols=conditions,
)
display(df1)

,Sample,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime,Cq_ref_GAPDH_mean,Cq_ref_GAPDH_std,Cq_ref_GAPDH_ste,Cq_test_CER1_mean,Cq_test_CER1_std,Cq_test_CER1_ste,deltaCq_CER1vGAPDH,Cq_test_LHX1_mean,Cq_test_LHX1_std,Cq_test_LHX1_ste,deltaCq_LHX1vGAPDH,Cq_test_BETA-ACTIN_mean,Cq_test_BETA-ACTIN_std,Cq_test_BETA-ACTIN_ste,deltaCq_BETA-ACTINvGAPDH,Cq_test_OCT4_mean,Cq_test_OCT4_std,Cq_test_OCT4_ste,deltaCq_OCT4vGAPDH,Cq_test_NANOG_mean,Cq_test_NANOG_std,Cq_test_NANOG_ste,deltaCq_NANOGvGAPDH,Cq_test_CYP26A1_mean,Cq_test_CYP26A1_std,Cq_test_CYP26A1_ste,deltaCq_CYP26A1vGAPDH,Cq_test_GAL_mean,Cq_test_GAL_std,Cq_test_GAL_ste,deltaCq_GALvGAPDH
0,P6_D0-2DD_SD0.4_TR1_0hr,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr,19.663406,0.054098,0.027049,27.788371,0.206532,0.103266,8.124964,33.791922,1.095024,0.547512,14.128516,17.526059,0.241492,0.120746,-2.137347,19.718873,0.105993,0.052997,0.055467,23.833878,0.146431,0.073216,4.170471,31.111532,0.338581,0.169291,11.448126,24.759354,0.046906,0.023453,5.095948
1,P6_D0-2DD_SD0.4_TR2_0hr,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr,19.978406,0.078841,0.039420,27.563875,0.205466,0.102733,7.585469,34.827203,1.045754,0.522877,14.848797,18.045148,0.232593,0.116296,-1.933258,20.006325,0.092618,0.046309,0.027919,24.262804,0.085936,0.042968,4.284398,31.842685,0.199607,0.099803,11.864279,23.862472,0.039719,0.019859,3.884066
2,P6_D0-2DD_SD0.46_TR1_0hr,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr,20.998449,0.279837,0.139919,28.213882,0.224956,0.112478,7.215433,35.967575,1.386451,0.693225,14.969126,18.681082,0.085936,0.042968,-2.317367,20.918691,0.053659,0.026829,-0.079759,25.197929,0.090311,0.045155,4.199479,31.843460,0.472867,0.236433,10.845010,24.477236,0.100120,0.050060,3.478786
3,P6_D0-2DD_SD0.46_TR2_0hr,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr,20.036676,0.142687,0.071344,27.709924,0.096847,0.048424,7.673248,34.465282,1.518305,0.759152,14.428606,17.802394,0.125761,0.062881,-2.234282,19.866225,0.073478,0.036739,-0.170451,24.548445,0.039807,0.019903,4.511769,32.052568,0.503606,0.251803,12.015892,23.985244,0.164027,0.082013,3.948568
4,P12_D0-2DD_SD0.4_TR1_0hr,P12_D0-2DD_SD0.4_0hr,P12_D0-2DD_SD0.4,P12_SD0.4,0hr,18.746962,0.097374,0.048687,26.945137,0.153794,0.076897,8.198175,33.596881,1.652812,0.826406,14.849919,16.864871,0.122249,0.061124,-1.882091,19.151731,0.097165,0.048582,0.404768,23.468493,0.109923,0.054962,4.721531,31.227805,0.234125,0.117062,12.480842,24.463710,0.114597,0.057299,5.716748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,P6_D2-6DD_SD0.46_TR2_48hr,P6_D2-6DD_SD0.46_48hr,P6_D2-6DD_SD0.46,P6_SD0.46,48hr,19.536572,0.039869,0.019935,23.066997,0.035627,0.017814,3.530425,23.558110,0.100217,0.050108,4.021538,17.847068,0.071014,0.035507,-1.689503,21.391172,0.082538,0.041269,1.854600,26.044573,0.032733,0.016367,6.508001,21.884355,0.043829,0.021915,2.347784,23.793566,0.047298,0.023649,4.256994
28,P12_D2-6DD_SD0.4_TR1_48hr,P12_D2-6DD_SD0.4_48hr,P12_D2-6DD_SD0.4,P12_SD0.4,48hr,16.657511,0.032711,0.016355,20.589805,0.066980,0.033490,3.932294,22.597327,0.063306,0.031653,5.939815,16.181258,0.212503,0.106251,-0.476253,18.925966,0.041214,0.020607,2.268454,23.171407,0.341492,0.170746,6.513896,20.931437,0.057073,0.028536,4.273926,21.425192,0.138337,0.069169,4.767680
29,P12_D2-6DD_SD0.4_TR2_48hr,P12_D2-6DD_SD0.4_48hr,P12_D2-6DD_SD0.4,P12_SD0.4,48hr,17.222098,0.029493,0.014746,21.266656,0.149367,0.074684,4.044559,22.911227,0.069105,0.034552,5.689129,16.622411,0.008244,0.004122,-0.599686,19.605827,0.092706,0.046353,2.383729,23.716335,0.081905,0.040952,6.494237,21.023277,0.048619,0.024309,3.801179,21.987461,0.208063,0.104032,4.765363
30,P12_D2-6DD_SD0.5_TR1_48hr,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr,16.947210,0.039318,0.019659,22.062930,0.120814,0.060407,5.115720,25.177193,0.145714,0.072857,8.229983,21.933896,0.274933,0.137466,4.986686,19.493414,0.078976,0.039488,2.546204,2

#### Get Calibrators from control samples

In [8]:
df2 = get_deltaCq_stats(df1, biorep_col="bio_reps")
df_calibrators = get_calibrators(
    df2,
    ctrl_col="ctrl_calibrator",
    condition_col="cond_sd",
    assign_ctrl_samples=[
        "P6_D0-2DD_SD0.4",
        "P6_D0-2DD_SD0.46",
        "P12_D0-2DD_SD0.4",
        "P12_D0-2DD_SD0.5",
    ],
    assign_cond_group=[
        "P6_SD0.4",
        "P6_SD0.46",
        "P12_SD0.4",
        "P12_SD0.5",
    ],
)
with displaydf_full():
  display(df_calibrators)

,deltaCq_CER1vGAPDH,deltaCq_LHX1vGAPDH,deltaCq_BETA-ACTINvGAPDH,deltaCq_OCT4vGAPDH,deltaCq_NANOGvGAPDH,deltaCq_CYP26A1vGAPDH,deltaCq_GALvGAPDH
P6_SD0.4,7.855217,14.488657,-2.035303,0.041693,4.227435,11.656202,4.490007
P6_SD0.46,7.444340,14.698866,-2.275825,-0.125105,4.355624,11.430451,3.713677
P12_SD0.4,7.882852,15.102660,-1.847673,0.298518,4.404494,12.004857,5.221106
P12_SD0.5,7.632371,13.818526,-2.032231,0.010748,4.017147,11.683972,4.877240


#### Get foldchange by deltadeltaCq Method

In [9]:
df3 = get_deltadeltaCqMethod_foldchange(
    df1, df_calibrators, biorep_col="bio_reps", condition_col="cond_sd"
)

with displaydf_full():
  display(df3)

,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime,2^(deltadeltaCq_CER1vGAPDH)_mean,2^(deltadeltaCq_CER1vGAPDH)_std,2^(deltadeltaCq_CER1vGAPDH)_ste,2^(deltadeltaCq_CER1vGAPDH)_ci95_upper,2^(deltadeltaCq_CER1vGAPDH)_ci95_lower,2^(deltadeltaCq_LHX1vGAPDH)_mean,2^(deltadeltaCq_LHX1vGAPDH)_std,2^(deltadeltaCq_LHX1vGAPDH)_ste,2^(deltadeltaCq_LHX1vGAPDH)_ci95_upper,2^(deltadeltaCq_LHX1vGAPDH)_ci95_lower,2^(deltadeltaCq_BETA-ACTINvGAPDH)_mean,2^(deltadeltaCq_BETA-ACTINvGAPDH)_std,2^(deltadeltaCq_BETA-ACTINvGAPDH)_ste,2^(deltadeltaCq_BETA-ACTINvGAPDH)_ci95_upper,2^(deltadeltaCq_BETA-ACTINvGAPDH)_ci95_lower,2^(deltadeltaCq_OCT4vGAPDH)_mean,2^(deltadeltaCq_OCT4vGAPDH)_std,2^(deltadeltaCq_OCT4vGAPDH)_ste,2^(deltadeltaCq_OCT4vGAPDH)_ci95_upper,2^(deltadeltaCq_OCT4vGAPDH)_ci95_lower,2^(deltadeltaCq_NANOGvGAPDH)_mean,2^(deltadeltaCq_NANOGvGAPDH)_std,2^(deltadeltaCq_NANOGvGAPDH)_ste,2^(deltadeltaCq_NANOGvGAPDH)_ci95_upper,2^(deltadeltaCq_NANOGvGAPDH)_ci95_lower,2^(deltadeltaCq_CYP26A1vGAPDH)_mean,2^(deltadeltaCq_CYP26A1vGAPDH)_std,2^(deltadeltaCq_CYP26A1vGAPDH)_ste,2^(deltadeltaCq_CYP26A1vGAPDH)_ci95_upper,2^(deltadeltaCq_CYP26A1vGAPDH)_ci95_lower,2^(deltadeltaCq_GALvGAPDH)_mean,2^(deltadeltaCq_GALvGAPDH)_std,2^(deltadeltaCq_GALvGAPDH)_ste,2^(deltadeltaCq_GALvGAPDH)_ci95_upper,2^(deltadeltaCq_GALvGAPDH)_ci95_lower
0,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr,1.017531,0.265966,0.188066,1.386141,0.648921,1.031320,0.356709,0.252231,1.525693,0.536947,1.002503,0.100113,0.070791,1.141253,0.863752,1.000046,0.013502,0.009548,1.018759,0.981332,1.000780,0.055853,0.039494,1.078188,0.923371,1.010419,0.204676,0.144728,1.294086,0.726752,1.089507,0.611597,0.432464,1.937137,0.241877
1,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr,1.012614,0.225331,0.159333,1.324907,0.700321,1.017598,0.266477,0.188428,1.386916,0.648279,1.000415,0.040728,0.028799,1.056861,0.943968,1.000494,0.044458,0.031437,1.062110,0.938878,1.005863,0.153361,0.108443,1.218411,0.793315,1.083472,0.589764,0.417026,1.900842,0.266101,1.013284,0.231272,0.163534,1.333811,0.692756
2,P12_D0-2DD_SD0.4_0hr,P12_D0-2DD_SD0.4,P12_SD0.4,0hr,1.023981,0.311565,0.220309,1.455787,0.592174,1.015384,0.249021,0.176084,1.360509,0.670260,1.000285,0.033742,0.023859,1.047049,0.953520,1.002713,0.104247,0.073714,1.147192,0.858234,1.024243,0.313286,0.221526,1.458435,0.590051,1.054922,0.475100,0.335946,1.713376,0.396467,1.059597,0.495472,0.350351,1.746286,0.372908
3,P12_D0-2DD_SD0.5_0hr,P12_D0-2DD_SD0.5,P12_SD0.5,0hr,1.005362,0.146651,0.103698,1.208611,0.802114,1.017853,0.268418,0.189800,1.389861,0.645844,1.010662,0.207063,0.146416,1.297637,0.723687,1.001742,0.083503,0.059046,1.117472,0.886012,1.005228,0.144799,0.102388,1.205909,0.804547,1.038031,0.393723,0.278404,1.583704,0.492359,1.070733,0.541239,0.382714,1.820852,0.320614
4,P6_D2-4DD_SD0.4_24hr,P6_D2-4DD_SD0.4,P6_SD0.4,24hr,54.522519,1.565384,1.106894,56.692031,52.353008,360.208403,52.625506,37.211852,433.143633,287.273172,0.957362,0.240722,0.170216,1.290986,0.623738,0.899489,0.101247,0.071593,1.039811,0.759167,0.892414,0.177386,0.125431,1.138259,0.646570,398.626235,23.188816,16.396969,430.764294,366.488176,3.357810,0.265134,0.187478,3.725267,2.990352
5,P6_D2-4DD_SD0.46_24hr,P6_D2-4DD_SD0.46,P6_SD0.46,24hr,125.754725,6.897661,4.877383,135.314396,116.195055,1103.610465,84.395832,59.676865,1220.577121,986.643810,1.030118,0.059657,0.042184,1.112798,0.947438,0.602145,0.087419,0.061814,0.723301,0.480989,0.867039,0.083943,0.059357,0.983378,0.750700,618.722440,123.137640,87.071460,789.382502,448.062378,2.126041,0.179354,0.126822,2.374613,1.877469
6,P12_D2-4DD_SD0.4_24hr,P12_D2-4DD_SD0.4,P12_SD0.4,24hr,19.202386,6.505699,4.600224,28.218825,10.185947,182.793880,101.490170,71.764388,323.452080,42.135680,0.924078,0.360071,0.254609,1.423110,0.425045,0.586849,0.124578,0.088090,0.759506,0.414193,0.813957,0.339534,0.240087,1.284527,0.343386,164.757232,77.931532,55.105915,272.764825,56.749640,2.561951,1.496784,1.058386,4.636388,0.487514
7,P12_D2-4DD_SD0.5_24hr,P12_D2-4DD_SD

#### Creating Graphs

##### WTC11 P6 Seeding Density 0.4M Definitive Endoderm Marker Gene

In [10]:
# TODO: Add this function to qpcr/utils/core.py
def rename_col(col_name: str) -> str:
    pattern = r"^2\^\(.*_(.*?)v.*\)_(.*)$"
    match = re.match(pattern, col_name)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    else:
        return col_name

# Extracting WTC11 P6 sample rows with Seeding Density 0.4M
df = df3.copy()
pattern = r"^P6_.*_SD0\.4_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

data = {
    "CHIR Timing": chirtimes,
    "CER1": df[primers[0]],
    "LHX1": df[primers[1]],
    "CYP26A": df[primers[5]],
    "GAL": df[primers[6]],
}
errorbars_upper = {
    "CER1": df[ci95_upper[0]],
    "LHX1": df[ci95_upper[1]],
    "CYP26A": df[ci95_upper[5]],
    "GAL": df[ci95_upper[6]],
}
errorbars_lower = {
    "CER1": df[ci95_lower[0]],
    "LHX1": df[ci95_lower[1]],
    "CYP26A": df[ci95_lower[5]],
    "GAL": df[ci95_lower[6]],
}
# Rename primers after data extraction from df
primers = [re.sub(r"_mean$", "", s) for s in primers]

# Creating the axis and labeling & setting dimensions
x_axis = [(chirtime, primer) for chirtime in chirtimes for primer in primers[:2] + primers[5:]]
foldchange = sum(zip(data["CER1"], data["LHX1"], data["CYP26A"], data["GAL"]), ())
errorbar_u = sum(zip(errorbars_upper["CER1"], errorbars_upper["LHX1"], errorbars_upper["CYP26A"], errorbars_upper["GAL"]), ())
errorbar_l = sum(zip(errorbars_lower["CER1"], errorbars_lower["LHX1"], errorbars_lower["CYP26A"], errorbars_lower["GAL"]), ())
source = ColumnDataSource(data=dict(x=x_axis, values=foldchange, upper=errorbar_u, lower=errorbar_l))

# Creating error bars
error = Whisker(base="x", upper="upper", lower="lower", source=source, level="annotation", line_width=1)
error.upper_head.size=10
error.lower_head.size=10

# Plotting the figure
p = figure(x_range=FactorRange(*x_axis), height=350, title="Relative fold change of DE marker genes in WTC11 P6 cells",
           toolbar_location=None, tools="", y_axis_type="log")

p.vbar(x="x", top="values", bottom=0.1, width=0.9, source=source,
      fill_color=factor_cmap("x", palette=Bright6, factors=primers[:2] + primers[5:], start=1, end=2))
p.add_layout(error)

p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

# Export directory
# exportfilename = "JR250129_qpcr_P6_SD0.4_D0.png"
# source = QPCRPATHS.ROOT/ "analysis" / exportfilename
# destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename
# 
# p.output_backend = "png"
# export_svgs(p, filename=exportfilename)
# shutil.move(str(source), str(destination))

show(p)

##### [Alternative] WTC11 P6 Seeding Density 0.4M Definitive Endoderm Marker Gene

In [11]:
# Extracting desired rows
df = df3.copy()
pattern = r"^P12_.*_SD0\.5_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
data = {
    "CHIR Timing": chirtimes,
    "CER1": df[primers[0]],
    "CER1_upper": df[ci95_upper[0]],
    "CER1_lower": df[ci95_lower[0]],
    "LHX1": df[primers[1]],
    "LHX1_upper": df[ci95_upper[1]],
    "LHX1_lower": df[ci95_lower[1]],
    "CYP26A": df[primers[5]],
    "CYP26A_upper": df[ci95_upper[5]],
    "CYP26A_lower": df[ci95_lower[5]],
    "GAL": df[primers[6]],
    "GAL_upper": df[ci95_upper[6]],
    "GAL_lower": df[ci95_lower[6]],
}
chirtimes[0] = "D0"
primers = [re.sub(r"_mean$", "", s) for s in primers]
source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=chirtimes, y_range=(0.001, 5000), height=350, width=650,
    title="Relative fold change of DE marker genes (vs GAPDH) in WTC11 P12 cells @ 0.5M Seeding",
    toolbar_location=None, tools="", y_axis_type="log"
)

bar_width = 0.2
p.vbar(
    x=dodge("CHIR Timing", -0.3, range=p.x_range),
    top="CER1", bottom=0.1, width=bar_width, source=source,
    color=Bright6[0], legend_label="CER1"
)
p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="LHX1", bottom=0.1, width=bar_width, source=source,
    color=Bright6[1], legend_label="LHX1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="CYP26A", bottom=0.1, width=bar_width, source=source,
    color=Bright6[2], legend_label="CYP26A"
)

p.vbar(
    x=dodge("CHIR Timing", 0.3, range=p.x_range),
    top="GAL", bottom=0.1, width=bar_width, source=source,
    color=Bright6[3], legend_label="GAL"
)

cer1_err = Whisker(
    base=dodge("CHIR Timing", -0.3, range=p.x_range),
    upper='CER1_upper', lower='CER1_lower', source=source, level="overlay"
)
cer1_err.upper_head.size = 10
cer1_err.lower_head.size = 10
p.add_layout(cer1_err)

lhx1_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='LHX1_upper', lower='LHX1_lower', source=source, level="overlay"
)
lhx1_err.upper_head.size = 10
lhx1_err.lower_head.size = 10
p.add_layout(lhx1_err)


cyp_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='CYP26A_upper', lower='CYP26A_lower', source=source, level="overlay"
)
cyp_err.upper_head.size = 10
cyp_err.lower_head.size = 10
p.add_layout(cyp_err)


gal_err = Whisker(
    base=dodge("CHIR Timing", 0.3, range=p.x_range),
    upper='GAL_upper', lower='GAL_lower', source=source, level="overlay"
)
gal_err.upper_head.size = 10
gal_err.lower_head.size = 10
p.add_layout(gal_err)

p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "vertical"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "Total time in +CHIR factor supplemented diff. media"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 


# Export directory
exportfilename = "JR250129_qpcr_P12_SD0.5_D2.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 

In [ ]:
# Extracting WTC11 P6 sample rows with Seeding Density 0.4M
df = df3.copy()
pattern = r"^P12_.*_SD0\.5_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]


primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
data = {
    "CHIR Timing": chirtimes,
    "OCT4": df[primers[3]],
    "OCT4_upper": df[ci95_upper[3]],
    "OCT4_lower": df[ci95_lower[3]],
    "NANOG": df[primers[4]],
    "NANOG_upper": df[ci95_upper[4]],
    "NANOG_lower": df[ci95_lower[4]],
}
chirtimes[0] = "D0"
primers = [re.sub(r"_mean$", "", s) for s in primers]
source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=chirtimes, y_range=(0.001, 1.3), height=350, width=350,
    title="Relative fold change of iPSC marker genes \n (vs GAPDH) in WTC11 P12 cells @ 0.5M Seeding",
    toolbar_location=None, tools=""
)

bar_width = 0.2
p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="OCT4", bottom=0.1, width=bar_width, source=source,
    color=Bright6[4], legend_label="OCT4"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="NANOG", bottom=0.1, width=bar_width, source=source,
    color=Bright6[5], legend_label="NANOG"
)

oct4_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='OCT4_upper', lower='OCT4_lower', source=source, level="overlay"
)
oct4_err.upper_head.size = 10
oct4_err.lower_head.size = 10
p.add_layout(oct4_err)

nano_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='NANOG_upper', lower='NANOG_lower', source=source, level="overlay"
)
nano_err.upper_head.size = 10
nano_err.lower_head.size = 10
p.add_layout(nano_err)

p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "Total time in +CHIR factor supplemented diff. media"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 

# Export directory
exportfilename = "JR250129_qpcr_P12_SD0.5_D0.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 

#### Compared to Multiome V4

In [13]:
# Extracting desired rows
df = df3.copy()
pattern = r"^P6_.*_SD0\.4_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting with pattern 2
df1 = df3.copy()
pattern1 = r"^P6_.*_SD0\.46_.*"
df1 = df1[df1.bio_reps.str.match(pattern1, na=False)]
df1 = df1.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
chirtimes[0] = "D0"
data = {
    "CHIR Timing": chirtimes[:2],
    "CER1_4": df[primers[0]].head(2),
    "CER1_4_upper": df[ci95_upper[0]].head(2),
    "CER1_4_lower": df[ci95_lower[0]].head(2),
    "CER1_46": df1[primers[0]].head(2),
    "CER1_46_upper": df1[ci95_upper[0]].head(2),
    "CER1_46_lower": df1[ci95_lower[0]].head(2),
    "CER1_multiome": [1.02, 230.17],
    "CER1_multiome_upper": [1.02 + 1.96 * 0.12, 230.17 + 1.96 * 0.12],
    "CER1_multiome_lower": [1.02 - 1.96 * 0.12, 230.17 - 1.96 * 0.12],
    "CER1_pilot": [0.928, 0.623],
}

primers = [re.sub(r"_mean$", "", s) for s in primers]

source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=chirtimes[:2], y_range=(0.001, 5000), height=350, width=650,
    title="Relative fold change of DE marker genes against other samples",
    toolbar_location=None, tools="", y_axis_type="log"
)

bar_width = 0.2
p.vbar(
    x=dodge("CHIR Timing", -0.3, range=p.x_range),
    top="CER1_4", bottom=0.1, width=bar_width, source=source,
    color=Bright6[0], legend_label="SSCT CER1 @0.4M"
)
p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="CER1_46", bottom=0.1, width=bar_width, source=source,
    color=Bright6[1], legend_label="SSCT CER1 @0.46M"
)
p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="CER1_multiome", bottom=0.1, width=bar_width, source=source,
    color=Bright6[2], legend_label="MULT CER1 @0.5M"
)
p.vbar(
    x=dodge("CHIR Timing", 0.3, range=p.x_range),
    top="CER1_pilot", bottom=0.1, width=bar_width, source=source,
    color=Bright6[3], legend_label="PILOT CER1 @0.5M"
)
# p.vbar(
#     x=dodge("CHIR Timing", 0, range=p.x_range),
#     top="LHX1", bottom=0.1, width=bar_width, source=source,
#     color=Bright6[1], legend_label="LHX1"
# )

# p.vbar(
#     x=dodge("CHIR Timing", 0.2, range=p.x_range),
#     top="CYP26A", bottom=0.1, width=bar_width, source=source,
#     color=Bright6[2], legend_label="CYP26A"
# )

cer1_err = Whisker(
    base=dodge("CHIR Timing", -0.3, range=p.x_range),
    upper='CER1_4_upper', lower='CER1_4_lower', source=source, level="overlay"
)
cer1_err.upper_head.size = 10
cer1_err.lower_head.size = 10
p.add_layout(cer1_err)

cer1_46_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='CER1_46_upper', lower='CER1_46_lower', source=source, level="overlay"
)
cer1_46_err.upper_head.size = 10
cer1_46_err.lower_head.size = 10
p.add_layout(cer1_46_err)

cer1_mult_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='CER1_multiome_upper', lower='CER1_multiome_lower', source=source, level="overlay"
)
cer1_mult_err.upper_head.size = 10
cer1_mult_err.lower_head.size = 10
p.add_layout(cer1_mult_err)

# lhx1_err = Whisker(
#     base=dodge("CHIR Timing", 0, range=p.x_range),
#     upper='LHX1_upper', lower='LHX1_lower', source=source, level="overlay"
# )
# lhx1_err.upper_head.size = 10
# lhx1_err.lower_head.size = 10
# p.add_layout(lhx1_err)


# cyp_err = Whisker(
#     base=dodge("CHIR Timing", 0.2, range=p.x_range),
#     upper='CYP26A_upper', lower='CYP26A_lower', source=source, level="overlay"
# )
# cyp_err.upper_head.size = 10
# cyp_err.lower_head.size = 10
# p.add_layout(cyp_err)


p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "vertical"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "Available iPSC -> DE qPCR data from diff. samples"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 


# Export directory
exportfilename = "JR250129_qpcr_compare_against_refs.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 

#### Compared to JR98

In [71]:
df = df3.copy()
pattern = r"^P6_.*_SD0\.4_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting with pattern 2
df1 = df3.copy()
pattern1 = r"^P6_.*_SD0\.46_.*"
df1 = df1[df1.bio_reps.str.match(pattern1, na=False)]
df1 = df1.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
chirtimes[0] = "D0"
chirtimes[1] = "D2"
data = {
    "CHIR Timing": chirtimes[:2],
    "CER1_4": df[primers[0]].head(2),
    "CER1_4_upper": df[ci95_upper[0]].head(2),
    "CER1_4_lower": df[ci95_lower[0]].head(2),
    "CER1_46": df1[primers[0]].head(2),
    "CER1_46_upper": df1[ci95_upper[0]].head(2),
    "CER1_46_lower": df1[ci95_lower[0]].head(2),
    "CER1_multiome": [1.02, 230.17],
    "CER1_multiome_upper": [1.02 + 1.96 * 0.12, 230.17 + 1.96 * 0.12],
    "CER1_multiome_lower": [1.02 - 1.96 * 0.12, 230.17 - 1.96 * 0.12],
    "CER1_pilot": [0.928, 0.623],
    "CER1_JR98_pilot_unperturbed": [1.0088341275309403, 12.020859782500551],
    "CER1_JR98_pilot_unperturbed_upper": [1.0088341275309403 + 1.96 * 0.13321522762474852, 12.020859782500551 + 1.96 * 2.8702048750496],
    "CER1_JR98_pilot_unperturbed_lower": [1.0088341275309403 - 1.96 * 0.13321522762474852, 12.020859782500551 - 1.96 * 2.8702048750496],
    "CER1_JR98_pilot_100vv": [1.0238170493310064, 1.954094178016728],
    "CER1_JR98_pilot_100vv_upper": [1.0238170493310064 + 1.96 * 0.31048784356508485, 1.954094178016728 + 1.96 * 0],
    "CER1_JR98_pilot_100vv_lower": [1.0238170493310064 - 1.96 * 0.31048784356508485, 1.954094178016728 - 1.96 * 0],
    "CER1_JR98_pilot_150vv": [1.0030149150386019, 5.817906052506323],
    "CER1_JR98_pilot_150vv_upper": [1.0030149150386019 + 1.96 * 0.0777104870007503, 5.817906052506323 + 1.96 * 0],
    "CER1_JR98_pilot_150vv_lower": [1.0030149150386019 - 1.96 * 0.0777104870007503, 5.817906052506323 - 1.96 * 0],
    "CER1_JR98_pilot_r2_100vv":       [1.0008649400831935, 84.6829721541199],
    "CER1_JR98_pilot_r2_100vv_upper": [1.0008649400831935 + 1.96 * 0.31048784356508485, 84.6829721541199 + 1.96 * 0],
    "CER1_JR98_pilot_r2_100vv_lower": [1.0008649400831935 - 1.96 * 0.31048784356508485, 84.6829721541199 - 1.96 * 0],
    "CER1_JR98_pilot_r2_150vv":       [1.162001688808762, 118.04965376667181],
    "CER1_JR98_pilot_r2_150vv_upper": [1.162001688808762 + 1.96 * 0.0777104870007503, 118.04965376667181 + 1.96 * 0],
    "CER1_JR98_pilot_r2_150vv_lower": [1.162001688808762 - 1.96 * 0.0777104870007503, 118.04965376667181 - 1.96 * 0],
}

primers = [re.sub(r"_mean$", "", s) for s in primers]

source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=chirtimes[:2], y_range=(0.001, 5000), height=350, width=650,
    title="Relative fold change of CER1 DE marker genes against other samples",
    toolbar_location=None, tools="", y_axis_type="log"
)
p.add_layout(Legend(), "right")

bar_width = 0.1
p.vbar(
    x=dodge("CHIR Timing", -0.4, range=p.x_range),
    top="CER1_4", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][0], legend_label="SSCT @0.4M"
)
p.vbar(
    x=dodge("CHIR Timing", -0.3, range=p.x_range),
    top="CER1_46", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][1], legend_label="SSCT @0.46M"
)
p.vbar(
    x=dodge("CHIR Timing", -0.2, range=p.x_range),
    top="CER1_multiome", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][2], legend_label="MULT @0.5M"
)
p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="CER1_pilot", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][3], legend_label="PILOT @0.5M"
)

p.vbar(
    x=dodge("CHIR Timing", 0, range=p.x_range),
    top="CER1_JR98_pilot_unperturbed", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][4], legend_label="JR98-Unperturb"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="CER1_JR98_pilot_100vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][5], legend_label="JR98-100vv R1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.2, range=p.x_range),
    top="CER1_JR98_pilot_150vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][6], legend_label="JR98-150vv R1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.3, range=p.x_range),
    top="CER1_JR98_pilot_r2_100vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][7], legend_label="JR98-100vv R2"
)

p.vbar(
    x=dodge("CHIR Timing", 0.4, range=p.x_range),
    top="CER1_JR98_pilot_r2_150vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][8], legend_label="JR98-150vv R2"
)

cer1_err = Whisker(
    base=dodge("CHIR Timing", -0.4, range=p.x_range),
    upper='CER1_4_upper', lower='CER1_4_lower', source=source, level="overlay"
)
cer1_err.upper_head.size = 10
cer1_err.lower_head.size = 10
p.add_layout(cer1_err)

cer1_46_err = Whisker(
    base=dodge("CHIR Timing", -0.3, range=p.x_range),
    upper='CER1_46_upper', lower='CER1_46_lower', source=source, level="overlay"
)
cer1_46_err.upper_head.size = 10
cer1_46_err.lower_head.size = 10
p.add_layout(cer1_46_err)

cer1_mult_err = Whisker(
    base=dodge("CHIR Timing", -0.2, range=p.x_range),
    upper='CER1_multiome_upper', lower='CER1_multiome_lower', source=source, level="overlay"
)
cer1_mult_err.upper_head.size = 10
cer1_mult_err.lower_head.size = 10
p.add_layout(cer1_mult_err)

cer1_jr98_unpt_err = Whisker(
    base=dodge("CHIR Timing", 0, range=p.x_range),
    upper='CER1_JR98_pilot_unperturbed_upper', lower='CER1_JR98_pilot_unperturbed_lower', source=source, level="overlay"
)
cer1_jr98_unpt_err.upper_head.size = 10
cer1_jr98_unpt_err.lower_head.size = 10
p.add_layout(cer1_jr98_unpt_err)

cer1_jr98_100_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='CER1_JR98_pilot_100vv_upper', lower='CER1_JR98_pilot_100vv_lower', source=source, level="overlay"
)
cer1_jr98_100_err.upper_head.size = 10
cer1_jr98_100_err.lower_head.size = 10
p.add_layout(cer1_jr98_100_err)

cer1_jr98_150_err = Whisker(
    base=dodge("CHIR Timing", 0.2, range=p.x_range),
    upper='CER1_JR98_pilot_150vv_upper', lower='CER1_JR98_pilot_150vv_lower', source=source, level="overlay"
)
cer1_jr98_150_err.upper_head.size = 10
cer1_jr98_150_err.lower_head.size = 10
p.add_layout(cer1_jr98_150_err)

cer1_jr98_r2_100_err = Whisker(
    base=dodge("CHIR Timing", 0.3, range=p.x_range),
    upper='CER1_JR98_pilot_r2_100vv_upper', lower='CER1_JR98_pilot_r2_100vv_lower', source=source, level="overlay"
)
cer1_jr98_r2_100_err.upper_head.size = 10
cer1_jr98_r2_100_err.lower_head.size = 10
p.add_layout(cer1_jr98_r2_100_err)

cer1_jr98_r2_150_err = Whisker(
    base=dodge("CHIR Timing", 0.4, range=p.x_range),
    upper='CER1_JR98_pilot_r2_150vv_upper', lower='CER1_JR98_pilot_r2_150vv_lower', source=source, level="overlay"
)
cer1_jr98_r2_150_err.upper_head.size = 10
cer1_jr98_r2_150_err.lower_head.size = 10
p.add_layout(cer1_jr98_r2_150_err)


p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "Available iPSC -> DE qPCR data from diff. samples"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 


# Export directory
exportfilename = "JR250129_qpcr_cer1_compare_against_refs_updated_250327.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 

In [72]:
df = df3.copy()
pattern = r"^P6_.*_SD0\.4_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting with pattern 2
df1 = df3.copy()
pattern1 = r"^P6_.*_SD0\.46_.*"
df1 = df1[df1.bio_reps.str.match(pattern1, na=False)]
df1 = df1.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
chirtimes[0] = "D0"
chirtimes[1] = "D2"
data = {
    "CHIR Timing": chirtimes[:2],
    "CER1_4": df[primers[5]].head(2),
    "CER1_4_upper": df[ci95_upper[5]].head(2),
    "CER1_4_lower": df[ci95_lower[5]].head(2),
    "CER1_46": df1[primers[5]].head(2),
    "CER1_46_upper": df1[ci95_upper[5]].head(2),
    "CER1_46_lower": df1[ci95_lower[5]].head(2),
    "CER1_JR98_pilot_unperturbed":       [1.0219714745057011, 209.35358154335893],
    "CER1_JR98_pilot_unperturbed_upper": [1.4350885871655508, 389.2895162488213],
    "CER1_JR98_pilot_unperturbed_lower": [0.6088543618458515, 29.41764683789654],
    "CER1_JR98_pilot_100vv":             [1.0650904555592229, 211.6079471335316],
    "CER1_JR98_pilot_100vv_upper":       [1.7836855806577159, 211.6079471335316],
    "CER1_JR98_pilot_100vv_lower":       [0.34649533046073, 211.6079471335316],
    "CER1_JR98_pilot_150vv":          [1.9600611467122546, 455.63981222103735],
    "CER1_JR98_pilot_150vv_upper":    [5.264181511817896, 455.63981222103735],
    "CER1_JR98_pilot_150vv_lower":    [0.000000000000001, 455.63981222103735],
    "CER1_JR98_pilot_r2_100vv":       [1.0008649400831935, 84.6829721541199],
    "CER1_JR98_pilot_r2_100vv_upper": [1.0824025487687352, 84.6829721541199 ],
    "CER1_JR98_pilot_r2_100vv_lower": [0.9193273313976518, 84.6829721541199 ],
    "CER1_JR98_pilot_r2_150vv":       [1.162001688808762, 118.04965376667181],
    "CER1_JR98_pilot_r2_150vv_upper": [2.3219639415955613, 118.04965376667181 ],
    "CER1_JR98_pilot_r2_150vv_lower": [0.0020394360219628815, 118.04965376667181 ],
}

primers = [re.sub(r"_mean$", "", s) for s in primers]

source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=chirtimes[:2], y_range=(0.001, 5000), height=350, width=650,
    title="Relative fold change of CYP26A1 DE marker genes against other samples",
    toolbar_location=None, tools="", y_axis_type="log"
)
p.add_layout(Legend(), "right")

bar_width = 0.1
p.vbar(
    x=dodge("CHIR Timing", -0.3, range=p.x_range),
    top="CER1_4", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][0], legend_label="SSCT @0.4M"
)
p.vbar(
    x=dodge("CHIR Timing", -0.2, range=p.x_range),
    top="CER1_46", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][1], legend_label="SSCT @0.46M"
)

p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="CER1_JR98_pilot_unperturbed", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][4], legend_label="JR98-Unperturb"
)

p.vbar(
    x=dodge("CHIR Timing", 0, range=p.x_range),
    top="CER1_JR98_pilot_100vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][5], legend_label="JR98-100vv R1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="CER1_JR98_pilot_150vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][6], legend_label="JR98-150vv R1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.2, range=p.x_range),
    top="CER1_JR98_pilot_r2_100vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][7], legend_label="JR98-100vv R2"
)

p.vbar(
    x=dodge("CHIR Timing", 0.3, range=p.x_range),
    top="CER1_JR98_pilot_r2_150vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][8], legend_label="JR98-150vv R2"
)

cer1_err = Whisker(
    base=dodge("CHIR Timing", -0.3, range=p.x_range),
    upper='CER1_4_upper', lower='CER1_4_lower', source=source, level="overlay"
)
cer1_err.upper_head.size = 10
cer1_err.lower_head.size = 10
p.add_layout(cer1_err)

cer1_46_err = Whisker(
    base=dodge("CHIR Timing", -0.2, range=p.x_range),
    upper='CER1_46_upper', lower='CER1_46_lower', source=source, level="overlay"
)
cer1_46_err.upper_head.size = 10
cer1_46_err.lower_head.size = 10
p.add_layout(cer1_46_err)

cer1_jr98_unpt_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='CER1_JR98_pilot_unperturbed_upper', lower='CER1_JR98_pilot_unperturbed_lower', source=source, level="overlay"
)
cer1_jr98_unpt_err.upper_head.size = 10
cer1_jr98_unpt_err.lower_head.size = 10
p.add_layout(cer1_jr98_unpt_err)

cer1_jr98_100_err = Whisker(
    base=dodge("CHIR Timing", 0, range=p.x_range),
    upper='CER1_JR98_pilot_100vv_upper', lower='CER1_JR98_pilot_100vv_lower', source=source, level="overlay"
)
cer1_jr98_100_err.upper_head.size = 10
cer1_jr98_100_err.lower_head.size = 10
p.add_layout(cer1_jr98_100_err)

cer1_jr98_150_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='CER1_JR98_pilot_150vv_upper', lower='CER1_JR98_pilot_150vv_lower', source=source, level="overlay"
)
cer1_jr98_150_err.upper_head.size = 10
cer1_jr98_150_err.lower_head.size = 10
p.add_layout(cer1_jr98_150_err)

cer1_jr98_r2_100_err = Whisker(
    base=dodge("CHIR Timing", 0.2, range=p.x_range),
    upper='CER1_JR98_pilot_r2_100vv_upper', lower='CER1_JR98_pilot_r2_100vv_lower', source=source, level="overlay"
)
cer1_jr98_r2_100_err.upper_head.size = 10
cer1_jr98_r2_100_err.lower_head.size = 10
p.add_layout(cer1_jr98_r2_100_err)

cer1_jr98_r2_150_err = Whisker(
    base=dodge("CHIR Timing", 0.3, range=p.x_range),
    upper='CER1_JR98_pilot_r2_150vv_upper', lower='CER1_JR98_pilot_r2_150vv_lower', source=source, level="overlay"
)
cer1_jr98_r2_150_err.upper_head.size = 10
cer1_jr98_r2_150_err.lower_head.size = 10
p.add_layout(cer1_jr98_r2_150_err)


p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "Available iPSC -> DE qPCR data from diff. samples"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 


# Export directory
exportfilename = "JR250129_qpcr_cyp_compare_against_refs_updated_250327.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 

In [74]:
df = df3.copy()
pattern = r"^P6_.*_SD0\.4_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]
df = df.rename(columns=rename_col)

# Extracting with pattern 2
df1 = df3.copy()
pattern1 = r"^P6_.*_SD0\.46_.*"
df1 = df1[df1.bio_reps.str.match(pattern1, na=False)]
df1 = df1.rename(columns=rename_col)

# Extracting foldchange and cleaning up column names
foldchange_pattern = r".*_mean"
chirtimes = df["cond_chirtime"].tolist()
primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]

ci95_upper_pattern = r".*_ci95_upper"
ci95_lower_pattern = r".*_ci95_lower"
ci95_upper = [m.group(0) for m in (re.match(ci95_upper_pattern, col) for col in df.columns) if m is not None]
ci95_lower = [m.group(0) for m in (re.match(ci95_lower_pattern, col) for col in df.columns) if m is not None]

primers = [m.group(0) for m in (re.match(foldchange_pattern, col) for col in df.columns) if m is not None]
chirtimes[0] = "D0"
chirtimes[1] = "D2"
data = {
    "CHIR Timing": chirtimes[:2],
    "CER1_4": df[primers[1]].head(2),
    "CER1_4_upper": df[ci95_upper[1]].head(2),
    "CER1_4_lower": df[ci95_lower[1]].head(2),
    "CER1_46": df1[primers[1]].head(2),
    "CER1_46_upper": df1[ci95_upper[1]].head(2),
    "CER1_46_lower": df1[ci95_lower[1]].head(2),
    "CER1_JR98_pilot_unperturbed":       [1.0335185219496266, 14.251699470453989],
    "CER1_JR98_pilot_unperturbed_upper": [1.545226946810922, 16.17756947580294 ],
    "CER1_JR98_pilot_unperturbed_lower": [0.5218100970883311, 12.325829465105038 ],
    "CER1_JR98_pilot_100vv":             [1.002894938991818, 3.75028438582843 ],
    "CER1_JR98_pilot_100vv_upper":       [1.1521416825093869, 3.75028438582843 ],
    "CER1_JR98_pilot_100vv_lower":       [0.8536481954742492,   3.75028438582843 ],
    "CER1_JR98_pilot_150vv":             [1.2114954494438788, 5.558709544943035],
    "CER1_JR98_pilot_150vv_upper":       [2.551942335451537,  5.558709544943035],
    "CER1_JR98_pilot_150vv_lower":       [0.000000000000001,  5.558709544943035],
    "CER1_JR98_pilot_r2_100vv":          [1.0389512165777095, 2.4693391564528553],
    "CER1_JR98_pilot_r2_100vv_upper":    [1.5913083947272866, 2.4693391564528553  ],
    "CER1_JR98_pilot_r2_100vv_lower":    [0.4865940384281324, 2.4693391564528553  ],
    "CER1_JR98_pilot_r2_150vv":          [1.0021008553675173,  7.5057695720053665],
    "CER1_JR98_pilot_r2_150vv_upper":    [1.1292159497490872, 7.5057695720053665],
    "CER1_JR98_pilot_r2_150vv_lower":    [0.8749857609859474, 7.5057695720053665],
}

primers = [re.sub(r"_mean$", "", s) for s in primers]

source = ColumnDataSource(data=data)

# Plotting the figure
p = figure(
    x_range=chirtimes[:2], y_range=(0.001, 5000), height=350, width=650,
    title="Relative fold change of LHX1 DE marker genes against other samples",
    toolbar_location=None, tools="", y_axis_type="log"
)
p.add_layout(Legend(), "right")

bar_width = 0.1
p.vbar(
    x=dodge("CHIR Timing", -0.3, range=p.x_range),
    top="CER1_4", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][0], legend_label="SSCT @0.4M"
)
p.vbar(
    x=dodge("CHIR Timing", -0.2, range=p.x_range),
    top="CER1_46", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][1], legend_label="SSCT @0.46M"
)

p.vbar(
    x=dodge("CHIR Timing", -0.1, range=p.x_range),
    top="CER1_JR98_pilot_unperturbed", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][4], legend_label="JR98-Unperturb"
)

p.vbar(
    x=dodge("CHIR Timing", 0, range=p.x_range),
    top="CER1_JR98_pilot_100vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][5], legend_label="JR98-100vv R1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.1, range=p.x_range),
    top="CER1_JR98_pilot_150vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][6], legend_label="JR98-150vv R1"
)

p.vbar(
    x=dodge("CHIR Timing", 0.2, range=p.x_range),
    top="CER1_JR98_pilot_r2_100vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][7], legend_label="JR98-100vv R2"
)

p.vbar(
    x=dodge("CHIR Timing", 0.3, range=p.x_range),
    top="CER1_JR98_pilot_r2_150vv", bottom=0.1, width=bar_width, source=source,
    color=Category10[10][8], legend_label="JR98-150vv R2"
)

cer1_err = Whisker(
    base=dodge("CHIR Timing", -0.3, range=p.x_range),
    upper='CER1_4_upper', lower='CER1_4_lower', source=source, level="overlay"
)
cer1_err.upper_head.size = 10
cer1_err.lower_head.size = 10
p.add_layout(cer1_err)

cer1_46_err = Whisker(
    base=dodge("CHIR Timing", -0.2, range=p.x_range),
    upper='CER1_46_upper', lower='CER1_46_lower', source=source, level="overlay"
)
cer1_46_err.upper_head.size = 10
cer1_46_err.lower_head.size = 10
p.add_layout(cer1_46_err)

cer1_jr98_unpt_err = Whisker(
    base=dodge("CHIR Timing", -0.1, range=p.x_range),
    upper='CER1_JR98_pilot_unperturbed_upper', lower='CER1_JR98_pilot_unperturbed_lower', source=source, level="overlay"
)
cer1_jr98_unpt_err.upper_head.size = 10
cer1_jr98_unpt_err.lower_head.size = 10
p.add_layout(cer1_jr98_unpt_err)

cer1_jr98_100_err = Whisker(
    base=dodge("CHIR Timing", 0, range=p.x_range),
    upper='CER1_JR98_pilot_100vv_upper', lower='CER1_JR98_pilot_100vv_lower', source=source, level="overlay"
)
cer1_jr98_100_err.upper_head.size = 10
cer1_jr98_100_err.lower_head.size = 10
p.add_layout(cer1_jr98_100_err)

cer1_jr98_150_err = Whisker(
    base=dodge("CHIR Timing", 0.1, range=p.x_range),
    upper='CER1_JR98_pilot_150vv_upper', lower='CER1_JR98_pilot_150vv_lower', source=source, level="overlay"
)
cer1_jr98_150_err.upper_head.size = 10
cer1_jr98_150_err.lower_head.size = 10
p.add_layout(cer1_jr98_150_err)

cer1_jr98_r2_100_err = Whisker(
    base=dodge("CHIR Timing", 0.2, range=p.x_range),
    upper='CER1_JR98_pilot_r2_100vv_upper', lower='CER1_JR98_pilot_r2_100vv_lower', source=source, level="overlay"
)
cer1_jr98_r2_100_err.upper_head.size = 10
cer1_jr98_r2_100_err.lower_head.size = 10
p.add_layout(cer1_jr98_r2_100_err)

cer1_jr98_r2_150_err = Whisker(
    base=dodge("CHIR Timing", 0.3, range=p.x_range),
    upper='CER1_JR98_pilot_r2_150vv_upper', lower='CER1_JR98_pilot_r2_150vv_lower', source=source, level="overlay"
)
cer1_jr98_r2_150_err.upper_head.size = 10
cer1_jr98_r2_150_err.lower_head.size = 10
p.add_layout(cer1_jr98_r2_150_err)


p.y_range.start = 0.1
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label = "Available iPSC -> DE qPCR data from diff. samples"
p.yaxis.axis_label = "Relative fold change wrt D0 (2^(-ΔΔCq))" 


# Export directory
exportfilename = "JR250129_qpcr_lxh1_compare_against_refs_updated_250327.png"
source = QPCRPATHS.ROOT/ "analysis" / exportfilename
destination = QPCRPATHS.DATADIR / experiment_id / "250129_qpcr_analysis" / "plots" / exportfilename

export_png(p, filename=exportfilename)
shutil.move(str(source), str(destination))

show(p) 